In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import random
from model_structure import get_preprocessing_transforms,BCNN, train_model, evaluate_model

In [2]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 60
INPUT_SIZE = 224
model_number = 2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Get transforms
train_transform, val_transform = get_preprocessing_transforms(INPUT_SIZE)

In [5]:
# Set data set directory
train_dir = f'data4model_{model_number}/train/'
val_dir = f'data4model_{model_number}/test/'

In [6]:
train_dataset = datasets.ImageFolder(
        root=train_dir,
        transform=train_transform
    )
    
val_dataset = datasets.ImageFolder(
        root=val_dir,
        transform=val_transform
    )

In [7]:
# Create data loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=4,
    pin_memory=True
)

In [8]:
model = BCNN(input_channels=3).to(DEVICE)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [9]:
summary(model, (3, INPUT_SIZE, INPUT_SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
              ReLU-2         [-1, 32, 224, 224]               0
         MaxPool2d-3         [-1, 32, 112, 112]               0
         Dropout2d-4         [-1, 32, 112, 112]               0
            Conv2d-5         [-1, 64, 112, 112]          18,496
              ReLU-6         [-1, 64, 112, 112]               0
         MaxPool2d-7           [-1, 64, 56, 56]               0
         Dropout2d-8           [-1, 64, 56, 56]               0
            Conv2d-9          [-1, 128, 56, 56]          73,856
             ReLU-10          [-1, 128, 56, 56]               0
        MaxPool2d-11          [-1, 128, 28, 28]               0
        Dropout2d-12          [-1, 128, 28, 28]               0
          Flatten-13               [-1, 100352]               0
           Linear-14                  [

In [ ]:
tracker = train_model(model, train_loader, val_loader, criterion, optimizer, NUM_EPOCHS, DEVICE, model_number)